In [1]:
!pip install -q catboost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

from catboost import CatBoostRegressor

In [3]:
# Columns to load
dtypes = {
    'start_time':              'str',
    'end_time':                'str',
    'start station id':        'float32',
    'end station id':          'float32',
    'start station latitude':  'float32',
    'start station longitude': 'float32',
    'end station latitude':    'float32',
    'end station longitude':   'float32',
}

# Load the data
rides = pd.read_csv('../input/2018-niceride-tripdata.csv',
                    usecols=dtypes.keys(), 
                    dtype=dtypes)

In [4]:
rides.head()

,start_time,end_time,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude
0,2018-04-24 16:03:04.2700,2018-04-24 16:25:57.6010,170.0,44.992538,-93.270256,2.0,44.984894,-93.256554
1,2018-04-24 16:38:40.5210,2018-04-24 17:07:31.1070,2.0,44.984894,-93.256554,13.0,44.986088,-93.272461
2,2018-04-24 17:51:10.1450,2018-04-24 18:00:17.4510,13.0,44.986088,-93.272461,94.0,44.978210,-93.260231
3,2018-04-24 18:50:05.7390,2018-04-24 19:04:22.0360,94.0,44.978210,-93.260231,13.0,44.986088,-93.272461
4,2018-04-25 08:49:05.2720,2018-04-25 08:56:40.3060,13.0,44.986088,-93.272461,43.0,44.973839,-93.274544


Now we can compute the demand at each station per hour.  That is, the difference between the number of inbound and outbound rides.

In [5]:
# Don't use data w/ missing station IDs
rides.dropna(inplace=True)
rides['start station id'] = rides['start station id'].astype('uint16')
rides['end station id'] = rides['end station id'].astype('uint16')

# Compute what hour each ride occurred at
for col in ['start', 'end']:
    tcol = col+'_time'
    rides[tcol] = rides[tcol].str.slice(0, 19)
    rides[tcol] = pd.to_datetime(rides[tcol], utc=True,
                             format='%Y-%m-%d %H:%M:%S')
    rides[col+'_hourofyear'] = (24*rides[tcol].dt.dayofyear +
                                rides[tcol].dt.hour)
    
# Only use rides w/ valid station IDs
rides = rides[
    (rides['start station id']>1) &
    (rides['start station id']<227) &
    (rides['end station id']>1) &
    (rides['end station id']<227)
]

# Number of outbound rides per hour
outbound = rides[['start station id', 'start_hourofyear', 'start_time']]
outbound.columns = ['station_id', 'hourofyear', 'time']
outbound = outbound.pivot_table(values='time',
                                index='hourofyear',
                                columns='station_id',
                                aggfunc='count',
                                fill_value=0)

# Number of inbound rides per hour
inbound = rides[['end station id', 'end_hourofyear', 'end_time']]
inbound.columns = ['station_id', 'hourofyear', 'time']
inbound = inbound.pivot_table(values='time',
                                index='hourofyear',
                                columns='station_id',
                                aggfunc='count',
                                fill_value=0)

# Compute difference in demand per hour
hours = np.arange(2456, 7790)
outbound = outbound.reindex(hours, fill_value=0.0)
inbound = inbound.reindex(hours, fill_value=0.0)
hourly_demand = inbound - outbound

In [6]:
hourly_demand.head()

station_id,2,3,4,5,6,7,8,9,10,11,...,213,214,216,217,219,222,223,224,225,226
hourofyear,,,,,,,,,,,,,,,,,,,,,
2456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2460,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In order to forecast several hours out, we need training data not just on how the demand changes over the course of one-hour intervals, but also over longer intervals.  We'll be forecasting up to 12 hours out into the future, so we'll accumulate demand changes for 1-12 hours ahead.

In [7]:
# Compute the demand over different periods
demand = []
for lag in range(12):
    this_demand = (hourly_demand
        .rolling(window=lag+1)
        .sum()
        .shift(-lag)
        .stack()
        .rename('demand')
        .reset_index())
    this_demand['hours_out'] = lag+1
    demand += [this_demand]
    
# Concatenate into a single DataFrame
demand = pd.concat(demand, axis=0, ignore_index=True)
demand.dropna(inplace=True)

Now we have a dataframe with the demand at a given lag, per hour, at each station.  That is, the difference between the number of inbound and outbound rides: demand values >0 indicate more rides are coming *in* to that station than out over the course of a given number of hours out.

In [8]:
demand.sample(n=20)

,hourofyear,station_id,demand,hours_out
4439237,2886,217,0.0,5
6085017,5624,115,-1.0,6
129119,3088,210,0.0,1
10430514,5615,219,-1.0,10
6441735,7373,29,0.0,6
10302675,4989,70,0.0,10
12572041,5464,144,-1.0,12
5526445,2886,94,3.0,6
6185892,6119,2,0.0,6
7845848,3599,10,0.0,8


To make predictions about the demand, we'll need features off which to base those predictions.  We'll add time and station-related features.

In [9]:
# Create time features
demand['datetime'] = '2018-01-01 00:00:01'
demand['datetime'] = pd.to_datetime(demand['datetime'])
demand['datetime'] += pd.to_timedelta(demand['hourofyear'], unit='h')
demand['week_of_year'] = demand['datetime'].dt.weekofyear
demand['day_of_week'] = demand['datetime'].dt.dayofweek
demand['hour'] = demand['datetime'].dt.hour

# Create df of station locations by id
starts = rides[['start station id', 'start station latitude', 'start station longitude']]
ends = rides[['end station id', 'end station latitude', 'end station longitude']]
starts.columns = ['station_id', 'latitude', 'longitude']
ends.columns = ['station_id', 'latitude', 'longitude']
stations = pd.concat([starts, ends], axis=0, ignore_index=True).groupby(['station_id']).first()

# Create station features
demand = demand.merge(stations, on='station_id')

# Remove raw columns which are no longer needed
demand.drop(['hourofyear',
             'station_id',
             'datetime'], axis='columns', inplace=True)

# Cast everything to float32
demand = demand.astype('float32')

And now we have a pretty big table (13M rows) with demand over different periods, across time, for each station.

In [10]:
demand

,demand,hours_out,week_of_year,day_of_week,hour,latitude,longitude
0,0.0,1.0,15.0,4.0,8.0,44.984894,-93.256554
1,0.0,1.0,15.0,4.0,9.0,44.984894,-93.256554
2,0.0,1.0,15.0,4.0,10.0,44.984894,-93.256554
3,0.0,1.0,15.0,4.0,11.0,44.984894,-93.256554
4,0.0,1.0,15.0,4.0,12.0,44.984894,-93.256554
5,-2.0,1.0,15.0,4.0,13.0,44.984894,-93.256554
6,0.0,1.0,15.0,4.0,14.0,44.984894,-93.256554
7,0.0,1.0,15.0,4.0,15.0,44.984894,-93.256554
8,0.0,1.0,15.0,4.0,16.0,44.984894,-93.256554
9,0.0,1.0,15.0,4.0,17.0,44.984894,-93.256554


Let's save that processed training data to file.

In [11]:
# Save to a feather file
demand.to_feather('nice_ride_2018_demand.feather')

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Modeling

In [12]:
# Split into in- and dependent variables
y_train = demand['demand']
x_train = demand[[c for c in demand.columns if c !='demand']]

Now can train a CatBoost model on the data to predict the demand.

In [13]:
# Actually don't think you need to scale it since y is normal-ish?
"""
def catboost_model(prc):
    return Pipeline([
        ('scaler',    RobustScaler()),
        ('regressor', CatBoostRegressor(loss_function='Quantile:alpha='+str(prc)))
    ])

# Models to predict median and 90% conf interval
model_05 = catboost_model(0.05)
model_50 = catboost_model(0.5)
model_95 = catboost_model(0.95)
"""

# Models to predict median and 90% conf interval
model_05 = CatBoostRegressor(loss_function='Quantile:alpha=0.05', depth=9, verbose=False)
model_50 = CatBoostRegressor(loss_function='Quantile:alpha=0.5', depth=9, verbose=False)
model_95 = CatBoostRegressor(loss_function='Quantile:alpha=0.95', depth=9, verbose=False)

In [14]:
%%time

# TODO: how long to fit just one?
model_50.fit(x_train, y_train)

CPU times: user 1h 55min 36s, sys: 9min 4s, total: 2h 4min 40s
Wall time: 36min 38s


In [15]:
%%time

# Fit models
model_05.fit(x_train, y_train)
#model_50.fit(x_train, y_train)
model_95.fit(x_train, y_train)

CPU times: user 3h 47min 31s, sys: 17min 16s, total: 4h 4min 47s
Wall time: 1h 12min 8s


In [16]:
# Save models to file
model_05.save_model('catboost_05.dat')
model_50.save_model('catboost_50.dat')
model_95.save_model('catboost_95.dat')

In [17]:
# Load a model
#model = CatBoostRegressor()
#model.load_model('catboost_50.dat')